In [ ]:
!pip install vllm

# vLLM API Endpoints - Kapsamlı Request/Response Dokümanı


## İçindekiler

1. [OpenAI Compatible Endpoints](#openai-compatible-endpoints)
2. [Custom vLLM Endpoints](#custom-vllm-endpoints)
3. [Kullanım İpuçları](#kullanim-ipuclari)

---

## OpenAI Compatible Endpoints

### 1. Chat Completions API

**Endpoint:** `POST /v1/chat/completions`

**Açıklama:** Chat formatındaki mesajları işleyerek LLM'den yanıt alır. OpenAI Chat API ile tam uyumludur. Api dökümantasyonu: https://platform.openai.com/docs/api-reference/chat/create

#### Request Body

| Field               | Type          | Required | Açıklama                                                  |
| ------------------- | ------------- | -------- | --------------------------------------------------------- |
| `model`             | string        | ✓        | Kullanılacak model adı (örn: "openai/gpt-oss-20b")        |
| `messages`          | array         | ✓        | Konuşma mesajları dizisi                                  |
| `temperature`       | number        | ✗        | Sampling temperature (0.0-2.0, default: 1.0)              |
| `max_tokens`        | integer       | ✗        | Maksimum üretilecek token sayısı                          |
| `top_p`             | number        | ✗        | Nucleus sampling (0.0-1.0, default: 1.0)                  |
| `frequency_penalty` | number        | ✗        | Frequency penalty (-2.0 - 2.0, default: 0.0)              |
| `presence_penalty`  | number        | ✗        | Presence penalty (-2.0 - 2.0, default: 0.0)               |
| `stop`              | string/array  | ✗        | Stop sequence(s)                                          |
| `stream`            | boolean       | ✗        | Streaming response (default: false)                       |
| `tools`             | array         | ✗        | Function calling için tool tanımları                      |
| `tool_choice`       | string/object | ✗        | Tool seçim stratejisi ("auto", "none", veya belirli tool) |
| `extra_body`        | object        | ✗        | vLLM özel parametreleri                                   |

#### Messages Object (Request)

İstek gövdesindeki `messages` alanı, aşağıdaki rolleri içeren mesajlardan oluşur:

- `system`: Modelin davranışını tanımlar. `content` zorunlu string.
- `user`: Kullanıcı girişleri. `content` string veya (multi‑modal modellerde) parça dizisi.
- `assistant`: Önceki model yanıtları. `content` string. İsteğe bağlı `tool_calls` bulunabilir.
- `tool`: Bir tool/function çalıştırma sonucunu modele iletmek için kullanılır. `tool_call_id` zorunludur.

Örnekler:

```json
{ "role": "system", "content": "You are a helpful assistant." }

{ "role": "user", "content": "Merhaba!" }

{
  "role": "assistant",
  "content": null,
  "tool_calls": [
    {
      "id": "call_abc123",
      "type": "function",
      "function": {"name":"get_weather","arguments":"{\"city\":\"Ankara\"}"}
    }
  ]
}

{
  "role": "tool",
  "tool_call_id": "call_abc123",
  "content": "{\"temp\":29,\"unit\":\"C\"}"
}
```

#### Extra Body Parameters (vLLM Özel)

**vLLM Özel Uçların Dökümantasyonu:** https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html#chat-api_1

| Field                 | Type    | Default | Açıklama                                  |
| --------------------- | ------- | ------- | ----------------------------------------- |
| `top_k`               | integer | -1      | Top-k sampling                            |
| `min_p`               | number  | 0.0     | Minimum probability threshold             |
| `repetition_penalty`  | number  | 1.0     | Repetition penalty                        |
| `use_beam_search`     | boolean | false   | Beam search aktif/pasif                   |
| `best_of`             | integer | 1       | Beam search için kandidat sayısı          |
| `ignore_eos`          | boolean | false   | EOS token'ı yok say                       |
| `min_tokens`          | integer | 0       | Minimum token sayısı                      |
| `skip_special_tokens` | boolean | true    | Özel token'ları atla                      |
| `guided_json`         | object  | null    | JSON schema constraint                    |
| `guided_regex`        | string  | null    | Regex pattern constraint                  |
| `guided_choice`       | array   | null    | Seçim kısıtlaması                         |
| `guided_grammar`      | string  | null    | Grammar constraint                        |
| `priority`            | integer | 0       | Request priority (düşük = yüksek öncelik) |
| `request_id`          | string  | null    | Custom request ID                         |

#### Request Örneği

```json
{
  "model": "openai/gpt-oss-20b",
  "messages": [
    {
      "role": "system",
      "content": "You are a helpful assistant."
    },
    {
      "role": "user",
      "content": "Python'da bir HTTP server nasıl kurulur?"
    }
  ],
  "max_tokens": 150,
  "temperature": 0.7,
  "top_p": 0.9,
  "stream": false,
  "extra_body": {
    "top_k": 50,
    "repetition_penalty": 1.1
  }
}
```

#### Response Body (Non-streaming)

| Field          | Type    | Açıklama                 |
| -------------- | ------- | ------------------------ |
| `id`           | string  | Unique response ID       |
| `object`       | string  | "chat.completion"        |
| `created`      | integer | Unix timestamp           |
| `model`        | string  | Kullanılan model adı     |
| `choices`      | array   | Response seçenekleri     |
| `usage`        | object  | Token kullanım bilgileri |
| `service_tier` | string  | Service tier bilgisi     |

#### Choices Object

| Field           | Type        | Açıklama                                                                 |
| --------------- | ----------- | ------------------------------------------------------------------------ |
| `index`         | integer     | Choice index                                                             |
| `message`       | object      | Üretilen assistant mesajı (aşağıdaki **Message Object (Response)**)      |
| `finish_reason` | string/null | Olası değerler: `stop`, `length`, `tool_calls`, `content_filter`, `null` |
| `logprobs`      | object/null | `logprobs.content` listesi döner (aşağıya bakınız).                      |

**`logprobs` formatı (Chat Completions):**

`logprobs.content` bir dizi nesnedir. Her eleman aşağıdaki alanlara sahiptir:

- `token` (string): Üretilen token.
- `logprob` (number): Bu token'ın log-olasılığı.
- `bytes` (array\|null): Token'ın UTF‑8 byte karşılığı.
- `top_logprobs` (array): Aynı konumdaki en olası alternatif tokenların listesi; her eleman `{ token, logprob, bytes }` içerir.

#### Message Object (Response)

| Field        | Type        | Açıklama                                                   |
| ------------ | ----------- | ---------------------------------------------------------- |
| `role`       | string      | Her zaman `assistant`.                                     |
| `content`    | string/null | Üretilen metin içeriği. Tool çağrılarında `null` olabilir. |
| `tool_calls` | array/null  | Modelin çağrılmasını istediği fonksiyonları belirtir.      |

**`tool_calls` elemanı:**

Her eleman aşağıdaki şemayı izler:

```json
{
  "id": "call_abc123",
  "type": "function",
  "function": {
    "name": "get_weather",
    "arguments": "{\"city\":\"Ankara\"}"
  }
}
```

#### Usage Object

| Field               | Type    | Açıklama            |
| ------------------- | ------- | ------------------- |
| `prompt_tokens`     | integer | Input token sayısı  |
| `completion_tokens` | integer | Output token sayısı |
| `total_tokens`      | integer | Toplam token sayısı |

#### Response Örneği

```json
{
  "id": "chatcmpl-abc123",
  "object": "chat.completion",
  "created": 1692897427,
  "model": "openai/gpt-oss-20b",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Python'da HTTP server kurmak için..."
      },
      "finish_reason": "stop",
      "logprobs": null
    }
  ],
  "usage": {
    "prompt_tokens": 25,
    "completion_tokens": 150,
    "total_tokens": 175
  }
}
```

### 2. Embeddings API

**Endpoint:** `POST /v1/embeddings`

**Açıklama:** Text embedding'leri üretir.

#### Request Body

| Field             | Type         | Required | Açıklama                       |
| ----------------- | ------------ | -------- | ------------------------------ |
| `input`           | string/array | ✓        | Embedding'i alınacak text(ler) |
| `model`           | string       | ✓        | Embedding model adı            |
| `encoding_format` | string       | ✗        | "float" veya "base64"          |
| `dimensions`      | integer      | ✗        | Output dimension sayısı        |

#### Request Örneği

```json
{
  "input": ["Hello world", "How are you?"],
  "model": "sentence-transformers/all-MiniLM-L6-v2",
  "encoding_format": "float"
}
```

#### Response Örneği

```json
{
  "object": "list",
  "data": [
    {
      "object": "embedding",
      "embedding": [0.012, -0.034, 0.789, ...],
      "index": 0
    },
    {
      "object": "embedding",
      "embedding": [0.543, -0.012, 0.234, ...],
      "index": 1
    }
  ],
  "model": "sentence-transformers/all-MiniLM-L6-v2",
  "usage": {
    "prompt_tokens": 6,
    "total_tokens": 6
  }
}
```

### 3. Models API

**Endpoint:** `GET /v1/models`

**Açıklama:** Mevcut modelleri listeler.

#### Request

Query parameter yok, sadece GET request.

#### Response Örneği

```json
{
  "object": "list",
  "data": [
    {
      "id": "openai/gpt-oss-20b",
      "object": "model",
      "created": 1692897427,
      "owned_by": "vllm"
    }
  ]
}
```

---

## Custom vLLM Endpoints

### 1. Tokenize API

**Endpoint:** `POST /tokenize`

**Açıklama:** Text'i token ID'lere çevirir.

#### Request Body

| Field                | Type    | Required | Açıklama                       |
| -------------------- | ------- | -------- | ------------------------------ |
| `prompt`             | string  | ✓        | Tokenize edilecek metin/prompt |
| `add_special_tokens` | boolean | ✗        | Özel token'lar eklensin mi     |

#### Request Örneği

```json
{
  "prompt": "Hello world!",
  "add_special_tokens": true
}
```

#### Response Örneği

```json
{
  "tokens": [15496, 1917, 0],
  "count": 3
}
```

### 2. Detokenize API

**Endpoint:** `POST /detokenize`

**Açıklama:** Token ID'leri text'e çevirir.

#### Request Body

| Field                 | Type    | Required | Açıklama             |
| --------------------- | ------- | -------- | -------------------- |
| `tokens`              | array   | ✓        | Token ID dizisi      |
| `skip_special_tokens` | boolean | ✗        | Özel token'ları atla |

#### Request Örneği

```json
{
  "tokens": [15496, 1917, 0],
  "skip_special_tokens": true
}
```

#### Response Örneği

```json
{
  "prompt": "Hello world!"
}
```

### 3. Classification API

**Endpoint:** `POST /classify`

**Açıklama:** Text sınıflandırma işlemi.

#### Request Body

| Field   | Type         | Required | Açıklama                    |
| ------- | ------------ | -------- | --------------------------- |
| `model` | string       | ✓        | Classification model adı    |
| `input` | string/array | ✓        | Sınıflandırılacak text(ler) |

#### Request Örneği

```json
{
  "model": "jason9693/Qwen2.5-1.5B-apeach",
  "input": ["I love this!", "This is terrible"]
}
```

#### Response Örneği

```json
{
  "id": "classify-7c87cac407b749a6935d8c7ce2a8fba2",
  "object": "list",
  "created": 1745383065,
  "model": "jason9693/Qwen2.5-1.5B-apeach",
  "data": [
    {
      "index": 0,
      "label": "Default",
      "probs": [
        0.565970778465271,
        0.4340292513370514
      ],
      "num_classes": 2
    },
    {
      "index": 1,
      "label": "Spoiled",
      "probs": [
        0.26448777318000793,
        0.7355121970176697
      ],
      "num_classes": 2
    }
  ],
  "usage": {
    "prompt_tokens": 20,
    "total_tokens": 20,
    "completion_tokens": 0,
    "prompt_tokens_details": null
  }
}
```

### 4. Score API

**Endpoint:** `POST /score`

**Açıklama:** Cross-encoder ile iki text arasında similarity score.

#### Request Body

| Field    | Type   | Required | Açıklama                |
| -------- | ------ | -------- | ----------------------- |
| `model`  | string | ✓        | Cross-encoder model adı |
| `text_1` | string | ✓        | İlk text                |
| `text_2` | string | ✓        | İkinci text             |

#### Request Örneği

```json
{
  "model": "cross-encoder/ms-marco-MiniLM-L-6-v2",
  "text_1": "What is machine learning?",
  "text_2": "Machine learning is a subset of artificial intelligence"
}
```

#### Response Örneği

```json
{
  "id": "score-request-id",
  "object": "list",
  "created": 693447,
  "model": "cross-encoder/ms-marco-MiniLM-L-6-v2",
  "data": [
    {
      "index": 0,
      "object": "score",
      "score": 1
    }
  ],
  "usage": {}
}
```

### 5. Re-rank API

**Endpoint:** `POST /v1/rerank`

**Açıklama:** Query'ye göre dökümanları yeniden sıralar.

#### Request Body

| Field              | Type    | Required | Açıklama                    |
| ------------------ | ------- | -------- | --------------------------- |
| `model`            | string  | ✓        | Re-rank model adı           |
| `query`            | string  | ✓        | Search query                |
| `documents`        | array   | ✓        | Sıralanacak dökümanlar      |
| `top_n`            | integer | ✗        | Döndürülecek maksimum sonuç |
| `return_documents` | boolean | ✗        | Döküman içeriğini döndür    |

#### Request Örneği

```json
{
  "model": "BAAI/bge-reranker-base",
  "query": "What is the capital of France?",
  "documents": [
    "The capital of Brazil is Brasilia.",
    "Paris is the capital and most populous city of France.",
    "The Eiffel Tower is located in Paris."
  ],
  "top_n": 2,
  "return_documents": true
}
```

#### Response Örneği

```json
{
  "id": "rerank-fae51b2b664d4ed38f5969b612edff77",
  "model": "BAAI/bge-reranker-base",
  "usage": {
    "total_tokens": 56
  },
  "results": [
    {
      "index": 1,
      "document": {
        "text": "The capital of France is Paris."
      },
      "relevance_score": 0.99853515625
    },
    {
      "index": 0,
      "document": {
        "text": "The capital of Brazil is Brasilia."
      },
      "relevance_score": 0.0005860328674316406
    }
  ]
}
```

## Kullanım İpuçları

### 1. Authentication

API Key'i request header'ında gönder:

```bash
curl -H "Authorization: Bearer your-api-key" \
     -H "Content-Type: application/json" \
     -X POST \
     http://localhost:8000/v1/chat/completions
```

### 2. Streaming Responses

Streaming için `Accept` header'ını ekle:

```bash
curl -H "Accept: text/event-stream" \
     -H "Authorization: Bearer your-api-key" \
     -H "Content-Type: application/json" \
     -d '{"stream": true, ...}' \
     http://localhost:8000/v1/chat/completions
```

### 3. Function Calling

Tool tanımı ve kullanımı:

```json
{
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "get_weather",
        "description": "Hava durumu bilgisi al",
        "parameters": {
          "type": "object",
          "properties": {
            "city": { "type": "string", "description": "Şehir adı" }
          },
          "required": ["city"]
        }
      }
    }
  ],
  "tool_choice": "auto"
}
```


# vLLM Instancelarını Nasıl Kapatırım?

Birden fazla **vLLM instance** aynı anda ayaklanmışsa (özellikle aralarında `generator` tipi process varsa), bunlar çakışmalara sebep olabilir. Bu durumda ilgili process ID’lerini (PID) bulup kapatmak gerekir.

### Çalışan vLLM process’lerini listelemek
```bash
komut1: ps aux | grep vllm
komut1_alternatif: lsof -i:8000

örnek çıktı: user     12345  2.3  5.6 12345678 456789 ? Sl   12:00   0:05 python -m vllm.entrypoints.openai.api_server --model Qwen/Qwen2.5-7B-Instruct

komut2: kill -9 12345
```

Bu komutlardan sonra sorun çözülmezse çalışma zamanını tekrar başlatın.

In [ ]:
#!lsof -i:8000

In [ ]:
#!ps aux | grep vllm

root        3736  8.1  2.8 12608884 1577208 ?    Sl   14:10   0:19 python3 -m vllm.entrypoints.openai.api_server --model Qwen/Qwen2.5-7B-Instruct --task generate --host 0.0.0.0 --port 8000 --dtype float16 --max-num-seqs 8 --enable-auto-tool-choice --tool-call-parser hermes
root        4939  0.0  0.0   7376  3440 ?        S    14:14   0:00 /bin/bash -c ps aux | grep vllm
root        5015  0.0  0.0   6484  2320 ?        S    14:14   0:00 grep vllm


In [ ]:
#!kill -9 123456

# Qwen/Qwen2.5-7B-Instruct LLM Modeli

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
  --model Qwen/Qwen2.5-7B-Instruct \
  --task generate \
  --host 0.0.0.0 \
  --port 8000 \
  --dtype float16 --gpu-memory-utilization 0.9 --max-num-seqs 8 \
  --enable-auto-tool-choice \
  --tool-call-parser hermes \
  > generator.log 2>&1 &


In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="dummy"
)


## Modelleri Listele — GET /v1/models
client.models.list() çağrısı bu uç noktayı kullanır.

In [ ]:
models = client.models.list()

for m in models.data:
    print(m.id)
    MODEL = m.id

Qwen/Qwen2.5-7B-Instruct


## Chat — POST /v1/chat/completions Vanilla

In [ ]:
messages = [
    {"role": "system", "content": "You are a concise assistant."},
    {"role": "user", "content": "Bu API ne işe yarar? Bir cümlede özetle."},
]

try:
    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        max_completion_tokens=500,
        temperature=0.3
    )
    choice = resp.choices[0]
    print("finish_reason:", choice.finish_reason)
    if resp.usage:
        print("usage:", resp.usage.model_dump())
    print("\nAssistant:")
    print(choice.message.content)
except Exception as e:
    print("Chat completions (temel) hatası:", e)


finish_reason: stop
usage: {'completion_tokens': 33, 'prompt_tokens': 35, 'total_tokens': 68, 'completion_tokens_details': None, 'prompt_tokens_details': None}

Assistant:
Bu API, belirli bir fonksiyon veya hizmeti otomatikleştirerek veya programlı olarak erişmek için kullanılır.


## Chat — POST /v1/chat/completions Streaming

In [ ]:
import sys

messages = [
    {"role": "system", "content": "You are a concise assistant."},
    {"role": "user", "content": "chat/completions ile bir streaming örneğini gösterir misin? 2-3 cümlelik bir yanıt üret."},
]

full_text = []
try:
    stream = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True,
        temperature=0.3,
        max_tokens=200,
    )

    for event in stream:
        if getattr(event, "choices", None):
            delta = event.choices[0].delta
            content = getattr(delta, "content", None)
            if content:
                full_text.append(content)
                sys.stdout.write(content)
                sys.stdout.flush()
            if getattr(event.choices[0], "finish_reason", None):
                print("\n[finish_reason]", event.choices[0].finish_reason)

    # Tam metni sonradan kullanmak istersen:
    final_text = "".join(full_text)
except Exception as e:
    print("Streaming chat hatası:", e)

Tabii, bir streaming örneği olarak: "Başlıyoruz... Yerel zamanlı hava durumu güncellemelerini alabilmek için chat/completions API'sini streaming moduyla çağırabiliriz."
[finish_reason] stop


## Chat — POST /v1/chat/completions (response_format)

In [ ]:
messages = [
    {"role": "system", "content": "You are a data API that returns JSON only. Always return a single valid JSON object with the requested fields and nothing else."},
    {"role": "user", "content": "Şu metinden {title, keywords[]} alanlarını JSON olarak çıkar ve sadece geçerli bir JSON nesnesi döndür: 'Agentic AI ile RAG mimarisini birleştiren bir demo hazırlayacağız.'"},
]

try:
    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        response_format={"type": "json_object"},
        temperature=0.0,
        max_tokens=1000
    )
    print(resp.choices[0].message.content)  # Geçerli JSON string
except Exception as e:
    print("Structured JSON hatası:", e)


{
  "title": "Agentic AI ile RAG mimarisini birleştiren bir demo",
  "keywords": ["Agentic AI", "RAG mimarisi", "demo"]
}


In [ ]:
messages = [
    {"role": "system", "content": "You are a linguist who teaches Turkish. Return JSON only. Always return a single valid JSON object with the requested arrays."},
    {"role": "user", "content": "Şu metinden {erkek[], kadın[]} alanlarını JSON olarak çıkar ve sadece geçerli bir JSON nesnesi döndür: 'Ayşe, Fatma, Esra, Batuhan, Mehmet, Ali'"},
]

try:
    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        response_format={"type": "json_object"},
        temperature=0.0,
        max_tokens=1000
    )
    print(resp.choices[0].message.content)  # Geçerli JSON string
except Exception as e:
    print("Structured JSON hatası:", e)


{"erkek":["Batuhan", "Mehmet", "Ali"], "kadın":["Ayşe", "Fatma", "Esra"]}


## Chat — POST /v1/chat/completions Function/Tool Calling

In [ ]:
import json
from typing import Optional, Dict, Any

# 1) Statik değer dönen fonksiyonu tanımla

def get_weather(city: str, date: Optional[str] = None) -> Dict[str, Any]:
    """Statik hava durumu döndürür (örnek amaçlı)."""
    return {
        "city": city,
        "date": date or "2025-08-18",
        "tempC": 45,
        "desc": "Güneşli",
    }

# 2) Fonksiyon adından gerçek Python fonksiyonuna giden mapping
function_registry = {
    "get_weather": get_weather,
}

# 3) Tool şemasını ve konuşmayı yeniden tanımla (self-contained olsun)
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Returns weather based on given city and date informations. Date is not neccessary.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string"},
                    "date": {"type": "string", "description": "YYYY-MM-DD"},
                },
                "required": ["city"],
            },
        },
    }
]

messages = [
    {"role": "system", "content": "You are a weather assistant. Use tool_call if you think that is needed."},
    {"role": "user", "content": "Ankara için bugün hava nasıl?"},
]

# 4) İlk çağrı: model gerek görürse tool_call isteği yollar
try:
    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        temperature=0,
        #store=True,
    )
    msg = resp.choices[0].message
    print("finish_reason:", resp.choices[0].finish_reason)

    if not getattr(msg, "tool_calls", None):
        print("Tool çağrısı yok, yanıt:")
        print(msg.content)
    else:
        # 5) Tool çağrılarını çalıştır ve sonucu modele geri ver
        for tc in msg.tool_calls:
            tool_name = tc.function.name
            raw_args = tc.function.arguments or "{}"
            try:
                args = json.loads(raw_args)
            except Exception:
                # Bazı sunucular JSON string yerine düz metin verebilir
                # Basit bir fallback: city'i ham metinden çıkaramıyorsak boş geçelim
                args = {}

            fn = function_registry.get(tool_name)
            if not fn:
                print(f"Tanımsız tool: {tool_name}")
                continue

            try:
                result = fn(**args)
            except TypeError:
                # Eksik/extra argüman varsa sadece beklenenleri geçir
                safe = {}
                for p in ("city", "date"):
                    if p in args:
                        safe[p] = args[p]
                result = fn(**safe)

            # 6) Tool sonucunu modele geri gönder
            followup = client.chat.completions.create(
                model=MODEL,
                messages=[
                    *messages,
                    msg,  # tool çağrısını içeren assistant mesajı
                    {
                        "role": "tool",
                        "tool_call_id": tc.id,
                        "content": json.dumps(result, ensure_ascii=False),
                    },
                ],
                temperature=0,
            )
            print("\nFinal assistant:")
            print(followup.choices[0].message.content)
except Exception as e:
    print("Tool calling (mapped) hatası:", e)

finish_reason: tool_calls

Final assistant:
Ankara'da bugün hava çok sıcak ve güneşli. Sıcaklık 45°C'e ulaşacak. Dışarıda uzun süre kalmanızı önermeyebiliriz.


## /tokenize

In [ ]:
import requests

URL = "http://localhost:8000/tokenize"

payload = {
    "prompt": "Merhaba dünya! Bu bir tokenizasyon testidir."
}

res = requests.post(URL, json=payload, timeout=30)
res.raise_for_status()

tok = res.json()
tok['tokens']
print("🧩 Tokenize sonucu:", tok['tokens'])



🧩 Tokenize sonucu: [26716, 10573, 64, 129463, 0, 27227, 15248, 3950, 449, 6405, 263, 1273, 307, 404, 13]


## /detokenize

In [ ]:
URL = "http://localhost:8000/detokenize"  # vLLM Detokenizer API
# input_ids: tokenize çağrısından aldığın ID'ler
payload = {
    "tokens": [26716, 10573, 64, 129463, 0, 27227, 15248, 3950, 449, 6405, 263, 1273, 307, 404, 13]
    }
res = requests.post(URL, json=payload, timeout=30)
res.raise_for_status()

detok = res.json()
print("🧵 Detokenize sonucu:", detok)

🧵 Detokenize sonucu: {'prompt': 'Merhaba dünya! Bu bir tokenizasyon testidir.'}


# /v1/embeddings Embedding Modeli

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
  --model BAAI/bge-small-en-v1.5 \
  --task embed \
  --host 0.0.0.0 \
  --port 8001 > embedding.log 2>&1 &


In [ ]:
import requests

resp = requests.post(
    "http://localhost:8001/v1/embeddings",
    headers={"Content-Type": "application/json"},
    json={"input": "Hello World", "model": "BAAI/bge-small-en-v1.5"}
)

print("Vectoral representation of the input sentence:" ,resp.json()['data'][0]['embedding'])

Vectoral representation of the input sentence: [0.0151519775390625, -0.02252197265625, 0.00853729248046875, -0.07421875, 0.0038585662841796875, 0.0027217864990234375, -0.0312347412109375, 0.04461669921875, 0.04400634765625, -0.00785064697265625, -0.0252532958984375, -0.033355712890625, 0.0143890380859375, 0.046539306640625, 0.0085601806640625, -0.01617431640625, 0.007381439208984375, -0.01898193359375, -0.11480712890625, -0.0181884765625, 0.1263427734375, 0.0297088623046875, 0.025299072265625, -0.034210205078125, -0.04107666015625, 0.006603240966796875, 0.01036834716796875, 0.022430419921875, 0.004444122314453125, -0.1273193359375, -0.0161590576171875, -0.020416259765625, 0.0472412109375, 0.0115966796875, 0.068115234375, 0.007297515869140625, -0.017852783203125, 0.040771484375, -0.01030731201171875, 0.0237274169921875, 0.0106048583984375, -0.028594970703125, 0.00811767578125, -0.015167236328125, 0.0309600830078125, -0.06597900390625, -0.0220947265625, 0.05401611328125, 0.00252151489257

# /v1/rerank RE-RANK Modeli

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
  --model cross-encoder/ms-marco-MiniLM-L-6-v2 \
  --task score \
  --host 0.0.0.0 \
  --port 8002 \
  > reranker.log 2>&1 &

## /v1/rerank

In [ ]:
import requests

URL = "http://localhost:8002/v1/rerank"
api_key = "dummy"

payload = {
    "model": "cross-encoder/ms-marco-MiniLM-L-6-v2",
    "query": "Hassas ciltler için en uygun doğal bakım ürünü hangisi?",
    "top_n": 2,
    "documents": [
        "Aloe vera ve papatya içeren doğal cilt bakım kremi, hassas ciltler için özel olarak tasarlanmıştır. Tahrişi azaltır ve nem dengesini korur.",
        "Bu yılın makyaj trendleri cesur renkler ve parlak dokular üzerine yoğunlaşıyor. Neon eyeliner ve ışıltılı highlighter öne çıkıyor.",
        "Shea yağı ve jojoba yağı ile üretilmiş organik nemlendirici, hassas ciltleri yatıştırır ve uzun süreli koruma sağlar."
    ],
    "return_documents": True
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

response = requests.post(URL, headers=headers, json=payload)

if response.ok:
    data = response.json()
    results = data.get("results", [])

    # relevance_score'a göre sırala (yüksekten düşüğe)
    results_sorted = sorted(results, key=lambda x: x["relevance_score"], reverse=True)

    print("\n📌 Re-rank Sıralaması:")
    for i, item in enumerate(results_sorted, start=1):
        score = item["relevance_score"]
        doc_text = item["document"]["text"]
        print(f"{i}. (Skor: {score:.4f}) → {doc_text}")
else:
    print("Hata:", response.text)



📌 Re-rank Sıralaması:
1. (Skor: 4.6328) → Aloe vera ve papatya içeren doğal cilt bakım kremi, hassas ciltler için özel olarak tasarlanmıştır. Tahrişi azaltır ve nem dengesini korur.
2. (Skor: 0.1362) → Shea yağı ve jojoba yağı ile üretilmiş organik nemlendirici, hassas ciltleri yatıştırır ve uzun süreli koruma sağlar.


## /v1/score

In [ ]:
import requests

url = "http://127.0.0.1:8002/v1/score"

payload = {
    "model": "cross-encoder/ms-marco-MiniLM-L-6-v2",
    "encoding_format": "float",
    "text_1": "Hassas ciltler için en uygun doğal bakım ürünü hangisi?",
    "text_2": "Aloe vera ve papatya içeren doğal cilt bakım kremi, hassas ciltler için özel olarak tasarlanmıştır. Tahrişi azaltır ve nem dengesini korur.",
}

headers = {"Content-Type": "application/json", "accept": "application/json"}

res = requests.post(url, json=payload, headers=headers, timeout=30)
data = res.json()
score = data["data"][0]["score"]
text = payload["text_2"]  # karşılaştırılan cümle
print(f'(Skor: {score:.4f}) -> {text}')

(Skor: 4.6328) -> Aloe vera ve papatya içeren doğal cilt bakım kremi, hassas ciltler için özel olarak tasarlanmıştır. Tahrişi azaltır ve nem dengesini korur.


# /classify Classification Modeli

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
  --model unitary/multilingual-toxic-xlm-roberta \
  --task classify \
  --port 8003 \
  > classifier.log 2>&1 &

In [ ]:
import requests
from transformers import AutoConfig

url = "http://localhost:8003/classify"
MODEL = "unitary/multilingual-toxic-xlm-roberta"

inputs = [
    "I am going to find you, and I am going to kill you",
    "What the hell is your problem man?",
    "I love you so much"
]

# 1) id2label (etiket isimleri) al
try:
    cfg = AutoConfig.from_pretrained(MODEL)
    id2label = {int(k): v for k, v in getattr(cfg, "id2label", {}).items()}
except Exception:
    id2label = {}

# 2) vLLM'e isteği at
payload = {"model": MODEL, "task": "classify", "input": inputs}
resp = requests.post(url, json=payload, timeout=30)
resp.raise_for_status()
out = resp.json()

# 3) Sonuçları yazdır
for item in out.get("data", []):
    idx = item["index"]
    probs = item["probs"]   # burada sadece 1 değer var: toxic ihtimali
    toxic_prob = probs[0]

    print(f"\n📌 Input[{idx}]: \"{inputs[idx]}\"")
    print(f"Probability of being TOXIC = {toxic_prob:.4f} ({toxic_prob*100:.1f}%)")


📌 Input[0]: "I am going to find you, and I am going to kill you"
Probability of being TOXIC = 0.9922 (99.2%)

📌 Input[1]: "What the hell is your problem man?"
Probability of being TOXIC = 0.7744 (77.4%)

📌 Input[2]: "I love you so much"
Probability of being TOXIC = 0.0005 (0.0%)
